In [1]:
!which python

/Users/apabook/Desktop/ds-protocol/static/tutorial/venv/bin/python


In [2]:
import json
import requests

# Complete Dataspace Flow


## Consts
Configuration of constant variables used throughout the notebook, including agent URLs and participant IDs.


In [3]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_provider_dsp_endpoint = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

# Dataspace setup
Authority is created and Consumer Agent and Provider Agent are registered in the Authority. Then, both participants onboard themselves in the system.


## Current Participants Information


In [4]:
url = data_space_provider + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:provider",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "saved_at": "2026-02-11T11:23:57.816363",
    "last_interaction": "2026-02-11T11:23:57.816419",
    "is_me": true
}


In [5]:
url = data_space_consumer + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:consumer",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2026-02-11T11:24:52.837929",
    "last_interaction": "2026-02-11T11:24:52.837985",
    "is_me": true
}


## Provider and Consumer IDs
Credential exchange and token retrieval for API calls.


In [6]:
url = data_space_provider + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_consumer_participant_id:
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

In [7]:
url = data_space_consumer + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_provider_participant_id:
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

In [8]:
print("Provider DID: {}\n".format(data_space_provider_participant_id))
print("Provider token: {}\n".format(data_space_provider_participant_token))
print("Consumer DID: {}\n".format(data_space_consumer_participant_id))
print("Consumer token: {}".format(data_space_consumer_participant_token))

Provider DID: did:jwk:provider

Provider token: b2yNqK0z5_IjrYp09UyL7OOjI1vxKgtgAEHnlf0qns0

Consumer DID: did:jwk:consumer

Consumer token: b2yNqK0z5_IjrYp09UyL7OOjI1vxKgtgAEHnlf0qns0


# Populate catalog

## Create Dataset, Distributions, Policies and Connector Instances
Provider is going to create a catalog, a data service, a dataset and a distribution to be offered to the consumer. The consumer will then discover the offer and accept it.


In [9]:
url = data_space_provider + "/api/v1/catalog-agent/catalogs/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "dctConformsTo": null,
    "dctCreator": null,
    "dctIdentifier": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dctIssued": "2026-02-11T11:25:15.903619Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainCatalog": true,
    "dspaceParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "foafHomePage": null,
    "id": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee"
}


In [10]:
url = data_space_provider + "/api/v1/catalog-agent/data-services/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_service_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dcatEndpointDescription": null,
    "dcatEndpointUrl": "http://127.0.0.1:1200/dsp/current",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db",
    "dctIssued": "2026-02-11T11:25:15.970263Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainDataService": true,
    "id": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db"
}


In [11]:
url = data_space_provider + "/api/v1/catalog-agent/datasets"
payload = {
    "dctTitle": "My dataspace dataset",
    "catalogId": catalog_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
    "dctIssued": "2026-02-11T11:27:19.881218Z",
    "dctModified": null,
    "dctTitle": "My dataspace dataset",
    "id": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2"
}


In [12]:
url = data_space_provider + "/api/v1/catalog-agent/distributions"
payload = {
    "dctTitle": "Distribution of my dataset",
    "dctDescription": "This is the distribution of my dataset",
    "dcatAccessService": data_service_id,
    "datasetId": dataset_id,
    "dctFormats": "http+asd"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "datasetId": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
    "dcatAccessService": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db",
    "dctDescription": "This is the distribution of my dataset",
    "dctFormat": "http+asd",
    "dctIssued": "2026-02-11T11:27:19.910239Z",
    "dctModified": null,
    "dctTitle": "Distribution of my dataset",
    "id": "urn:distribution:6c56c4bb-f1c1-4a98-a26c-0a20425f98ef"
}


## Create policies with template


In [13]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

[
    {
        "author": "",
        "content": {
            "permission": [
                {
                    "action": "use",
                    "constraint": [
                        {
                            "leftOperand": "bla",
                            "operator": "eq",
                            "rightOperand": "$bla"
                        }
                    ]
                }
            ]
        },
        "date": "2026-02-11T11:25:15.981207Z",
        "description": "Mi example super policy",
        "id": "policy-1",
        "parameters": {
            "$bla": {
                "dataType": "string",
                "restrictions": {},
                "ui": {
                    "hidden": false
                }
            }
        },
        "title": "Mi example policy",
        "version": "1.0"
    }
]


In [14]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates/instantiate-odrl-offer"
payload = {
    "id": "policy-1",
    "version": "1.0",
    "parameters": {
        "$bla": "did:web:hola.es"
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-11T11:27:19.945402Z",
    "entity": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:e2e600d0-d721-4410-8b56-a9807ae4c702",
    "instantiationParameters": {
        "$bla": "did:web:hola.es"
    },
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "bla",
                        "operator": "eq",
                        "rightOperand": "did:web:hola.es"
                    }
                ]
            }
        ]
    },
    "sourceTemplateId": "policy-1",
    "sourceTemplateVersion": "1.0"
}


## Or... just create policies with API


In [15]:
url = data_space_provider + "/api/v1/catalog-agent/odrl-policies"
payload = {
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }

        ],
        "obligation": [],
        "prohibition": []
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-11T11:27:19.965281Z",
    "entity": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:6e77a430-178e-4c55-8b86-320513704c1e",
    "instantiationParameters": null,
    "odrlOffer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "sourceTemplateId": null,
    "sourceTemplateVersion": null
}


## Create and configure connector to final system



In [16]:
url = data_space_provider + "/api/v1/connector/templates"
payload = {
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}"
        }
    },
    "parameters": [
        {
            "paramType": "STRING",
            "name": "ACCESS_URL",
            "title": "Url",
            "required": True
        },
        {
            "paramType": "VEC<STRING>",
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "required": True
        },
        {
            "paramType": "MAP<STRING,STRING>",
            "name": "ACCESS_HEADERS",
            "title": "Methods",
            "required": True
        }
    ]
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
connector_name = response_as_json["name"]
connector_version = response_as_json["version"]
print(json.dumps(response_as_json, indent=4))

{
    "name": "urn:connector-template:f0ae9938-6a6d-4c61-8346-11807fe265bc",
    "author": "admin",
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-11T11:27:19.982494Z",
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}",
            "bodyTemplate": null
        }
    },
    "parameters": [
        {
            "name": "ACCESS_URL",
            "title": "Url",
            "description": null,
            "paramType": "STRING",
            "required": true,
            "defaultValue": null,
            "autoFilled": false,
            "autoFilledType": null
        },
        {
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "description": null,
            "paramType": "VEC<STRIN

In [17]:
url = data_space_provider + "/api/v1/connector/instances"
payload = {
    "templateName": connector_name,
    "templateVersion": connector_version,
    "distributionId": distribution_id,
    "metadata": {
        "description": "asasdasddas",
        "ownerId": "asdas"
    },
    "dryRun": False,
    "parameters": {
        "ACCESS_URL": "https://dummyjson.com/products",
        "ACCESS_METHODS": ["GET", "HEAD"],
        "ACCESS_HEADERS": {
            "Content-type": "application/json"
        }
    }
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:connector-instance:6d45e610-45b2-4bf6-a1da-5bb42ce2c943",
    "name": "urn:connector-template:f0ae9938-6a6d-4c61-8346-11807fe265bc",
    "author": null,
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-11T11:27:20.003093Z",
    "authenticationConfig": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "https://dummyjson.com/products",
            "method": [
                "GET",
                "HEAD"
            ],
            "headers": {
                "Content-type": "application/json"
            },
            "bodyTemplate": null
        }
    },
    "distributionId": "urn:distribution:6c56c4bb-f1c1-4a98-a26c-0a20425f98ef"
}


## Consumer discovers the provider catalog and finds the offer


In [18]:
url = data_space_consumer + "/dsp/current/catalog/rpc/setup-catalog-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "filter": [], 
    "noCache": True
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["response"]["dataset"][-1]["hasPolicy"][0]["@id"]
target_id = response_as_json["response"]["dataset"][0]["@id"]
data_space_provider_dsp_endpoint = response_as_json["response"]["service"]["endpointURL"]
response_as_json


{'request': {'associatedAgentPeer': 'did:jwk:provider',
  'filter': [],
  'noCache': True},
 'response': {'@context': ['https://w3id.org/dspace/2025/1/context.jsonld'],
  '@type': 'Catalog',
  '@id': 'urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee',
  'identifier': 'urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee',
  'issued': '2026-02-11T11:25:15.903619',
  'catalog': [],
  'dataset': [{'@type': 'Dataset',
    '@id': 'urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2',
    'identifier': 'urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2',
    'issued': '2026-02-11T11:27:19.881218',
    'title': 'My dataspace dataset',
    'hasPolicy': [{'@id': 'urn:odrl-policy:e2e600d0-d721-4410-8b56-a9807ae4c702',
      'permission': [{'action': 'use',
        'constraint': [{'rightOperand': 'did:web:hola.es',
          'leftOperand': 'bla',
          'operator': 'eq'}]}],
      '@type': 'Offer'},
     {'@id': 'urn:odrl-policy:6e77a430-178e-4c55-8b86-320513704c1e',
      'permission': [{'action': 'u

In [19]:
print("Selected dataset target: \n{}\n".format(target_id))
print("Selected policy id: \n{}\n".format(policy_id))
print("Provider negotiation URL for DSP 2025: \n{}\n".format(data_space_provider_dsp_endpoint))


Selected dataset target: 
urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2

Selected policy id: 
urn:odrl-policy:434a9a44-4c50-432c-a4d9-b907f65c5b36

Provider negotiation URL for DSP 2025: 
http://127.0.0.1:1200/dsp/current



## Consumer discovers own callback address based on DSP version

In [20]:
url = data_space_consumer + "/.well-known/dspace-version/2025-1"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_dsp_endpoint = data_space_consumer + response_as_json["path"]
print("Consumer callback address: \n{}\n".format(data_space_consumer_dsp_endpoint))


Consumer callback address: 
http://127.0.0.1:1100/dsp/current



# Contract Negotiation

## Initialization of negotiation request (Consumer -> Provider)

In [72]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request-init"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"
            }
        ]
    }
}
try:
    response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
    response_as_json = response.json()
    cn_consumer_id = response_as_json["response"]["consumerPid"]
    cn_provider_id = response_as_json["response"]["providerPid"]
    print(json.dumps(response_as_json, indent=2))
except Exception as e:
    print("Error in response, {}.".format(e))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "offer": {
      "@id": "urn:odrl-policy:434a9a44-4c50-432c-a4d9-b907f65c5b36",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2"
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:c977ee2b-418e-402f-941f-8e2f90bd8262",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callback

## Provider creates initial offer (Provider -> Consumer)

In [73]:
# Provider creates initial offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,  # remove to test offer from provider
    "providerPid": cn_provider_id,  # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:434a9a44-4c50-432c-a4d9-b907f65c5b36",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2"
    },
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:322fa2f5-84b7-4004-900c-24a81acb436e",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": 

## Consumer sends negotiation request based on offer (Consumer -> Provider)

In [74]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:434a9a44-4c50-432c-a4d9-b907f65c5b36",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2"
    },
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:c977ee2b-418e-402f-941f-8e2f90bd8262",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http

## Provider updates/confirms the offer (Provider -> Consumer)

In [75]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:434a9a44-4c50-432c-a4d9-b907f65c5b36",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2"
    },
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:322fa2f5-84b7-4004-900c-24a81acb436e",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": 

## Consumer accepts the offer (Consumer -> Provider)

In [76]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-acceptance"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "ACCEPTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:c977ee2b-418e-402f-941f-8e2f90bd8262",
    "state": "ACCEPTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-12T21:08:45.263505Z",
    "updatedAt": "2026-02-12T21:08:46.886535Z",
    "identifiers": {
      "consumerPid": "urn:consumer-pid:6

## Provider creates the Agreement (Provider -> Consumer)

In [77]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-agreement"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "AGREED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:322fa2f5-84b7-4004-900c-24a81acb436e",
    "state": "AGREED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-12T21:08:45.222999Z",
    "updatedAt": "2026-02-12T21:08:47.247802Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:166e5

## Consumer verifies the agreement (Consumer -> Provider)

In [78]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-verification"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "VERIFIED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:c977ee2b-418e-402f-941f-8e2f90bd8262",
    "state": "VERIFIED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-12T21:08:45.263505Z",
    "updatedAt": "2026-02-12T21:08:47.610554Z",
    "identifiers": {
      "consumerPid": "urn:consumer-pid:6

## Provider finalizes the negotiation (Provider -> Consumer)

In [79]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-finalization"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    agreement = response_as_json
    agreement_id = response_as_json["negotiationAgentModel"]["agreement"]["id"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:692a4472-96fb-4466-9210-155670f7c79f",
    "providerPid": "urn:provider-pid:166e57d1-5192-4777-9990-9bac2f165c64",
    "state": "FINALIZED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:322fa2f5-84b7-4004-900c-24a81acb436e",
    "state": "FINALIZED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-12T21:08:45.222999Z",
    "updatedAt": "2026-02-12T21:08:47.964999Z",
    "identifiers": {
      "providerPid": "urn:provider-pid

## Final agreement

In [29]:
print("Final agreement: \n{}\n".format(json.dumps(agreement["negotiationAgentModel"]["agreement"], indent=2)))
print("Final agreement id: \n{}\n".format(agreement_id))

Final agreement: 
{
  "id": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
  "negotiationAgentProcessId": "urn:negotiation-process:0537ed3e-3339-4fba-8a46-b6bc2b41a0da",
  "negotiationAgentMessageId": "urn:negotiation-message:8be8aa5e-3fed-4ef0-b3f5-15d4de0478d4",
  "consumerParticipantId": "did:jwk:consumer",
  "providerParticipantId": "did:jwk:provider",
  "agreementContent": {
    "@id": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "@type": "Agreement",
    "assignee": "did:jwk:consumer",
    "assigner": "did:jwk:provider",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
    "timestamp": "1770809240"
  },
  "target": "urn:dataset:44a7d3cd-54a2-493e-851a-84db73b445d2",
  "state": "ACTIVE",
  "createdAt": "2026-02-11T11:27:20.906537Z",
  "updatedAt": "2026-02-11T11:27:21.183974Z"
}

Final agreement id: 
urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d



# Transfer Negotiation with Agents


## Transfer request setup (Consumer -> Provider)


In [41]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "agreementId": agreement_id,
    "format": "http+asd",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_process_consumer_pid = response_as_json["response"]["consumerPid"]
transfer_process_provider_pid = response_as_json["response"]["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "format": "http+asd",
    "dataAddress": null,
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6d442e8f-1999-4c66-bb65-97876efa2f08",
    "providerPid": "urn:provider-pid:addc33fe-4a3c-4ad9-9b80-502cbde2d7b9",
    "state": "REQUESTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:b0aad370-676e-4f90-97e0-5547093454d7",
    "state": "REQUESTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "callbackAddress": "http://127.0.0.1:

## Start transfer (Provider -> Consumer)

In [42]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    },
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6d442e8f-1999-4c66-bb65-97876efa2f08",
    "providerPid": "urn:provider-pid:addc33fe-4a3c-4ad9-9b80-502cbde2d7b9",
    "dataAddress": {
      "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
      "endpoint": "http://example.com",
      "endpointProperties": [
        {
          "name": "authorization",
          "value": "TOKEN-ABCDEFG"
        },
        {
          "name": "authType",
          "value": "bearer"
        }
      ]
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6d442e8f-1999-4c66-bb65-97876efa2f08",
    "providerPid": "urn:provider-pid:addc33fe-4a3c-4ad9-9b80-502cbde2d7b9",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:35b3c127-7cbd-483e-8c55-c5d7ecda37a9",
    "state": "STARTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "did:

## Suspend transfer (Consumer -> Provider)

In [43]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6d442e8f-1999-4c66-bb65-97876efa2f08",
    "providerPid": "urn:provider-pid:addc33fe-4a3c-4ad9-9b80-502cbde2d7b9",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6d442e8f-1999-4c66-bb65-97876efa2f08",
    "providerPid": "urn:provider-pid:addc33fe-4a3c-4ad9-9b80-502cbde2d7b9",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:b0aad370-676e-4f90-97e0-5547093454d7",
    "state": "SUSPENDED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
   

## Restart transfer (Consumer -> Provider)

In [33]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "dataAddress": null
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:e8f9eed3-6c38-4964-ac9d-b6310f046da3",
    "state": "STARTED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-

## Suspension by Provider (Provider -> Consumer)
(Note: Testing provider-initiated suspension)

In [34]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:25ec87aa-f85d-41b1-89cc-06af4d01931d",
    "state": "SUSPENDED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
   

## Failure Test: Attempt start with invalid parameters

In [35]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": None
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "dataAddress": null
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "5000",
    "reason": [
      "HTTP Error 400 Bad Request: {\"@context\":[\"https://w3id.org/dspace/2025/1/context.jsonld\"],\"@type\":\"TransferError\",\"consumerPid\":null,\"providerPid\":null,\"code\":\"6030\",\"reason\":[\"TransferProcessMessageType TransferStartMessage is not allowed here. Current state is SUSPENDED ByProvider\",\"Failed to parse file\"]}"
    ]
  }
}


## Failure Test: Attempt duplicate or invalid suspension

In [36]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "6030",
    "reason": [
      "TransferProcessMessageType TransferSuspensionMessage is not allowed here. Current state is SUSPENDED",
      "Failed to parse file"
    ]
  }
}


## Finalize transfer (Provider -> Consumer)

In [37]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-completion"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:ce323fd2-c90c-4048-82f4-a337cf733d3f",
    "providerPid": "urn:provider-pid:37800d8f-93d8-440c-8fd5-60dc07665961",
    "state": "COMPLETED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:25ec87aa-f85d-41b1-89cc-06af4d01931d",
    "state": "COMPLETED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:371ab6b7-2bdb-4534-a3bf-64b8f0d1f57d",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-11T11:27:21.237199Z",